In [1]:
!pip install -q ultralytics
import os, shutil, yaml
import pandas as pd
from sklearn.model_selection import KFold
from ultralytics import YOLO
import torch
import numpy

def convert_polygon_to_bbox(points):
    xs = points[::2]
    ys = points[1::2]
    xmin = min(xs)*512
    xmax = max(xs)*512
    ymin = min(ys)*512
    ymax = max(ys)*512

    x_center = (xmin + xmax) / 2
    y_center = (ymin + ymax) / 2
    width = xmax - xmin
    height = ymax - ymin

    return x_center, y_center, width, height

# === SETUP ===
all_images = []
for f in os.listdir(f"/kaggle/input/lacuna-solar-hugojoao/yolo/train/images"):
    if f.endswith(".png"):
        all_images.append(f"/kaggle/input/lacuna-solar-hugojoao/yolo/train/images/{f}")
for f in os.listdir(f"/kaggle/input/lacuna-solar-hugojoao/yolo/val/images"):
    if f.endswith(".png"):
        all_images.append(f"/kaggle/input/lacuna-solar-hugojoao/yolo/val/images/{f}")

# Folder for all images
os.makedirs("/kaggle/working/all_images", exist_ok=True)
for file_path in all_images:
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)
        dest_path = os.path.join("/kaggle/working/all_images", file_name)
        shutil.copy2(file_path, dest_path)
    else:
        print(f'File not found: {file_path}')

all_labels = []
for f in os.listdir(f"/kaggle/input/lacuna-solar-hugojoao/yolo/train/labels"):
    if f.endswith(".txt"):
        all_labels.append(f"/kaggle/input/lacuna-solar-hugojoao/yolo/train/labels/{f}")
for f in os.listdir(f"/kaggle/input/lacuna-solar-hugojoao/yolo/val/labels"):
    if f.endswith(".txt"):
        all_labels.append(f"/kaggle/input/lacuna-solar-hugojoao/yolo/val/labels/{f}")

# Folder for all labels
os.makedirs("/kaggle/working/all_labels1", exist_ok=True)
for file_path in all_labels:
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)
        dest_path = os.path.join("/kaggle/working/all_labels1", file_name)
        shutil.copy2(file_path, dest_path)
    else:
        print(f'File not found: {file_path}')

# change polygons to boxes
os.makedirs("/kaggle/working/all_labels", exist_ok=True)
labels = [x for x in os.listdir("/kaggle/working/all_labels1") if x.endswith(".txt")]
for label_path in labels:
    with open(f"/kaggle/working/all_labels1/{label_path}", "r") as f:
        lines = f.readlines()
    new_lines = []
    for line in lines:
        parts = list(map(float, line.strip().split()))
        class_id = int(parts[0])
        polygon = parts[1:]
        x_c, y_c, w, h = convert_polygon_to_bbox(polygon)
        # Normalize
        x_c /= 512
        w /= 512
        y_c /= 512
        h /= 512
        new_lines.append(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")
    # Save new label file
    output_path = f"/kaggle/working/all_labels/{label_path}"
    with open(output_path, "w") as f:
        f.writelines(new_lines)




df = pd.DataFrame({'image': all_images})
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)


# === FOLDING FOLDING FOLDING ===
for i, (train_idx, val_idx) in enumerate(kf.split(df)):
    if i > 2:
        continue

    #if i in [0,1]:
    #    continue

    print(f"Fold {i} de 2")

    os.makedirs(f"/kaggle/working/fold_{i}", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/train", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/val", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/train/images", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/val/images", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/train/labels", exist_ok=True)
    os.makedirs(f"/kaggle/working/fold_{i}/val/labels", exist_ok=True)


    # === COPY DATA ===
    for idx in train_idx:
        base = df.iloc[idx]["image"]
        shutil.copy(f"/kaggle/working/all_images/{os.path.basename(base)}",
                    f"/kaggle/working/fold_{i}/train/images/{os.path.basename(base)}")
        shutil.copy(f"/kaggle/working/all_labels/{os.path.basename(base).replace('.png', '.txt')}",
                    f"/kaggle/working/fold_{i}/train/labels/{os.path.basename(base).replace('.png', '.txt')}")
        
    for idx in val_idx:
        base = df.iloc[idx]["image"]
        shutil.copy(f"/kaggle/working/all_images/{os.path.basename(base)}",
                    f"/kaggle/working/fold_{i}/val/images/{os.path.basename(base)}")
        shutil.copy(f"/kaggle/working/all_labels/{os.path.basename(base).replace('.png', '.txt')}",
                    f"/kaggle/working/fold_{i}/val/labels/{os.path.basename(base).replace('.png', '.txt')}")
        

    # === CUSTOM DATA.YAML ===
    data_yaml = {
        'train': f"/kaggle/working/fold_{i}/train/images",
        'val': f"/kaggle/working/fold_{i}/val/images",
        'segmentation': "true",
        'names': {0: 'thermal', 1: 'photovoltaic'}
    }

    with open(f"/kaggle/working/fold_{i}/data.yaml", 'w') as f:
        yaml.dump(data_yaml, f)

    # Load YOLOv8 Model
    model = YOLO("yolo11l.pt")

    torch.cuda.empty_cache()
    
    # Train the model
    model.train(
        data= f"/kaggle/working/fold_{i}/data.yaml",
        epochs=150,
        imgsz=512,
        batch=16,
        augment=True,
        project="results",
        name=f'model_{i}',
        exist_ok=True,
        patience=20,
        lr0=1e-3,
        lrf = 0.1
        #mixup: 1.0,
        #copy_paste: 1.0,
    )

    torch.cuda.empty_cache()
    
    # predictions
    preds = {
        "img": [],
        "boil": [],
        "pan": [],
    }
    for img in [x for x in os.listdir(f"/kaggle/working/fold_{i}/train/images") if x.endswith(".png")]:

        results = model(f"/kaggle/working/fold_{i}/train/images/{img}")
        result = results[0]

        preds["img"].append(img)

        class_counts = {}
    
        for cls in result.boxes.cls.tolist():
            class_counts[int(cls)] = class_counts.get(int(cls), 0) + 1
    
        class_names = model.names
        named_counts = {class_names[k]: v for k, v in class_counts.items()}
    
        if "thermal" in named_counts:
            preds["boil"].append(named_counts["thermal"])
        else:
            preds["boil"].append(0)
            
        if "photovoltaic" in named_counts:
            preds["pan"].append(named_counts["photovoltaic"])
        else:
            preds["pan"].append(0)
    pd.DataFrame(preds).to_csv(f"fold_{i}_train.csv")

    torch.cuda.empty_cache()
    
    preds = {
        "img": [],
        "boil": [],
        "pan": [],
    }
    for img in [x for x in os.listdir(f"/kaggle/working/fold_{i}/val/images") if x.endswith(".png")]:

        results = model(f"/kaggle/working/fold_{i}/val/images/{img}")
        result = results[0]

        preds["img"].append(img)

        class_counts = {}
    
        for cls in result.boxes.cls.tolist():
            class_counts[int(cls)] = class_counts.get(int(cls), 0) + 1
    
        class_names = model.names
        named_counts = {class_names[k]: v for k, v in class_counts.items()}
    
        if "thermal" in named_counts:
            preds["boil"].append(named_counts["thermal"])
        else:
            preds["boil"].append(0)
            
        if "photovoltaic" in named_counts:
            preds["pan"].append(named_counts["photovoltaic"])
        else:
            preds["pan"].append(0)
    pd.DataFrame(preds).to_csv(f"fold_{i}_val.csv")

    shutil.rmtree(f"/kaggle/working/fold_{i}")

    torch.cuda.empty_cache()
        

shutil.rmtree("/kaggle/working/all_images")
shutil.rmtree("/kaggle/working/all_labels1")
shutil.rmtree("/kaggle/working/all_labels")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 15.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Fold 0 de 2


100%|██████████| 49.0M/49.0M [00:00<00:00, 229MB/s]


Ultralytics 8.3.107 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=/kaggle/working/fold_0/data.yaml, epochs=150, time=None, patience=20, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=results, name=model_0, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

100%|██████████| 755k/755k [00:00<00:00, 18.2MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytics

...


99 epochs completed in 2.052 hours.
Optimizer stripped from results/model_2/weights/last.pt, 51.2MB
Optimizer stripped from results/model_2/weights/best.pt, 51.2MB

Validating results/model_2/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11l summary (fused): 190 layers, 25,280,854 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:12<00:00,  1.14it/s]


                   all        422        920      0.888      0.738       0.81      0.487
               thermal         74         83      0.924      0.759      0.843      0.414
          photovoltaic        358        837      0.851      0.717      0.777      0.559


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 24.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to results/model_2

image 1/1 /kaggle/working/fold_2/train/images/IDdzTLIqJQ6tp03je.png: 512x512 1 photovoltaic, 44.7ms
Speed: 1.2ms preprocess, 44.7ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/working/fold_2/train/images/IDMHBK7.png: 512x512 2 photovoltaics, 46.4ms
Speed: 1.0ms preprocess, 46.4ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/working/fold_2/train/images/IDa9F8EILRh2cir.png: 512x512 4 photovoltaics, 45.2ms
Speed: 0.9ms preprocess, 45.2ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/working/fold_2/train/images/IDIll8HjsUXlNV.png: 512x512 1 photovoltaic, 45.3ms
Speed: 0.9ms preprocess, 45.3ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /kaggle/working/fold_2/train/images/IDoXyyG8htIz5.png: 512x512 2 photovoltaics, 45.3ms
Speed: 